In [ ]:
import numpy as np
import pandas as pd
from MVA_algo import *
from AnchorOptimalProjector import *
from sklearn.linear_model import LinearRegression, RidgeCV, MultiTaskLassoCV
from tqdm import tqdm
from matplotlib import pyplot as plt
from itertools import permutations
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, r2_score
import torch
from IRM_ICP import InvariantRiskMinimization, InvariantCausalPrediction
from IRM import IRMLinearRegression2 as IRMLinearRegression
from CVP import CVPLinearRegression, CVPLinearRegression2
from sklearn.cross_decomposition import PLSRegression

In [ ]:
data = pd.read_csv('../data/Air_Quality.csv')

In [ ]:
data_cleaned = data.loc[:, ~data.columns.str.contains('^Unnamed')][:9357]
data_cleaned['Date'] = pd.to_datetime(data_cleaned['Date'], format='%d/%m/%Y')

# Define a function to map month to season
def get_season(month):
    if month in [12, 1, 2]:
        return 0  # Winter
    elif month in [3, 4, 5]:
        return 1  # Spring
    elif month in [6, 7, 8]:
        return 2  # Summer
    elif month in [9, 10, 11]:
        return 3  # Autumn

# Apply the function to extract the season
data_cleaned['Season'] = data_cleaned['Date'].dt.month.map(get_season)

In [ ]:
Y = data_cleaned[['CO(GT)',	'PT08.S1(CO)',	'NMHC(GT)',	'C6H6(GT)',	'PT08.S2(NMHC)',	'NOx(GT)',	'PT08.S3(NOx)',	'NO2(GT)',	'PT08.S4(NO2)',	'PT08.S5(O3)']].to_numpy()
A = data_cleaned['Season'].to_numpy()[:, None]
X = data_cleaned[['T', 'RH', 'AH']].to_numpy()

In [ ]:
Y = (Y - Y.mean(axis=0))/Y.std(axis=0)
X = (X - X.mean(axis=0))/X.std(axis=0)

In [ ]:
idx_train = (A==0)[:, 0]
idx_val = ((A==3) | (A==1))[:, 0]
idx_test = (A==2)[:, 0]
X_train, X_test, X_val = X[idx_train, :], X[idx_test.T, :], X[idx_val, :]
A_train, A_test, A_val = A[idx_train, :], A[idx_test.T, :], A[idx_val, :]
Y_train, Y_test, Y_val = Y[idx_train, :], Y[idx_test.T, :], Y[idx_val, :]

In [7]:
gammas = np.logspace(-10, 10, 20)
lambdas = np.logspace(-3, 3, 20)
thetas = np.logspace(-3, 3, 20)
alphas = np.logspace(-10, 10, 20)
deltas = np.logspace(-10, 10, 20)
patience = 200
n_epochs = 50000
learning_rate = 0.1
n_components = [1, 2, 3]
score, score_ar, score_irm, score_cvp, score_pls_ar, score_ridge, score_ridge_anchor = [], [], [], [], [], [], []
seasons = np.array([0, 1, 2, 3])
combinations = list(permutations(seasons, 3))  # All train-val-test splits
lr = LinearRegression()
for train_season, val_season, test_season in tqdm(combinations):
    idx = [i not in list(permutations([0,1,2, 3], 3))[0] for i in np.array([0,1,2, 3])]
    val_season2 = seasons[idx]
    idx_train = (A==train_season | (A==val_season2))[:, 0]
    idx_val = (A==val_season)[:, 0]
    idx_test = (A==test_season)[:, 0]
    X_train, X_test, X_val = X[idx_train, :], X[idx_test.T, :], X[idx_val, :]
    A_train, A_test, A_val = A[idx_train, :], A[idx_test.T, :], A[idx_val, :]
    Y_train, Y_test, Y_val = Y[idx_train, :], Y[idx_test.T, :], Y[idx_val, :]

    # Selecting gamma opt
    
    mse_scores_ar = []
    for gamma in gammas:
        aop = AnchorOptimalProjection(gamma=gamma)
        X_anchor, Y_anchor = aop.fit_transform(A_train, X_train, Y_train)
        lr.fit(X_anchor, Y_anchor)
        Y_pred = lr.predict(X_val)
        mse_scores_ar.append(mean_squared_error(Y_val, Y_pred))
    gamma_opt = gammas[np.argmin(mse_scores_ar)]

    # gamma and ncp opt
    mse_scores_pls_ar = []
    for gamma in gammas:
        mse_scores_for_gamma = []
        for ncp in n_components:
            aop = AnchorOptimalProjection(gamma=gamma)
            X_anchor, Y_anchor = aop.fit_transform(A_train, X_train, Y_train)
            pls = PLSRegression(n_components=ncp)
            pls.fit(X_anchor, Y_anchor)
            Y_pred = pls.predict(X_val)
            mse = mean_squared_error(Y_val, Y_pred)
            mse_scores_for_gamma.append(mse)
        mse_scores_pls_ar.append(mse_scores_for_gamma)
    mse_scores_pls_ar = np.array(mse_scores_pls_ar)
    best_gamma_idx, best_ncp_idx = np.unravel_index(np.argmin(mse_scores_pls_ar), mse_scores_pls_ar.shape)
    gamma_opt_pls = gammas[best_gamma_idx]
    ncp_opt = n_components[best_ncp_idx]

    # alpha opt ridge
    mse_scores_ridge = []
    for alpha in alphas:
        ridge = Ridge(alpha=alpha)
        ridge.fit(X_train, Y_train)
        Y_pred = ridge.predict(X_val)
        mse_scores_ridge.append(mean_squared_error(Y_val, Y_pred))
    alpha_opt = alphas[np.argmin(mse_scores_ridge)]


    #  Selecting lambda opt for IRM
    input_dim = X_train.shape[1]
    output_dim = Y_train.shape[1]
    mse_scores_irm = []
    for l in lambdas:
        irm_model = IRMLinearRegression(input_dim, output_dim, lambda_irm=l, n_epochs=n_epochs, learning_rate=learning_rate, patience=patience, tol=1e-4, verbose=False)
        irm_model.fit(X_train, Y_train, A_train[:, 0])
        Y_pred = irm_model.predict(X_val)
        mse_scores_irm.append(mean_squared_error(Y_val, Y_val))
    lambda_opt = lambdas[np.argmin(mse_scores_irm)]

    # selecting theta opt for CVP
    mse_scores_cvp = []
    for theta in thetas:
        cvp_model = CVPLinearRegression(input_dim, output_dim, lambda_cvp=theta, n_epochs=n_epochs, learning_rate=learning_rate, patience=patience, tol=1e-4, verbose=False)
        cvp_model.fit(X_train, Y_train, A_train[:, 0])
        sklearn_model = cvp_model.set_sklearn_model()
        Y_pred = sklearn_model.predict(X_val)
        mse_scores_cvp.append(mean_squared_error(Y_val, Y_val))
    theta_opt = thetas[np.argmin(mse_scores_cvp)]

    # optimal anchor regression
    aop = AnchorOptimalProjection(gamma=gamma_opt)
    X_anchor, Y_anchor = aop.fit_transform(A_train, X_train, Y_train)
    lr.fit(X_anchor, Y_anchor)
    Y_pred = lr.predict(X_test)
    score_ar.append(mean_squared_error(Y_test, Y_pred))

    # optimal anchor PLS
    aop = AnchorOptimalProjection(gamma=gamma_opt_pls)
    X_anchor, Y_anchor = aop.fit_transform(A_train, X_train, Y_train)
    pls = PLSRegression(n_components=ncp_opt)
    pls.fit(X_anchor, Y_anchor)
    Y_pred = pls.predict(X_test)
    score_pls_ar.append(mean_squared_error(Y_test, Y_pred))

    # optimal IRM regression
    irm_model = IRMLinearRegression(input_dim, output_dim, lambda_irm=lambda_opt, n_epochs=n_epochs, learning_rate=learning_rate, patience=patience, tol=1e-4, verbose=False)
    irm_model.fit(X_train, Y_train, A_train[:, 0])
    Y_pred = irm_model.predict(X_test)
    score_irm.append(mean_squared_error(Y_test, Y_pred))


    # optimal CVP regression
    cvp_model = CVPLinearRegression(input_dim, output_dim, lambda_cvp=theta_opt, n_epochs=n_epochs, learning_rate=learning_rate, patience=patience, tol=1e-4, verbose=False)
    cvp_model.fit(X_train, Y_train, A_train[:, 0])
    sklearn_model = cvp_model.set_sklearn_model()
    Y_pred = sklearn_model.predict(X_test)
    score_cvp.append(mean_squared_error(Y_test, Y_pred))

    lr.fit(X_train, Y_train)
    Y_pred = lr.predict(X_test)
    score.append(mean_squared_error(Y_test, Y_pred))

    ridge = Ridge(alpha=alpha_opt)
    ridge.fit(X_train, Y_train)
    Y_pred = ridge.predict(X_test)
    score_ridge.append(mean_squared_error(Y_test, Y_pred))

    aop = AnchorOptimalProjection(gamma=gamma_opt)
    X_anchor, Y_anchor = aop.fit_transform(A_train, X_train, Y_train)
    ridge_anchor = Ridge(alpha=alpha_opt)
    ridge_anchor.fit(X_anchor, Y_train)
    Y_pred = ridge_anchor.predict(X_test)
    score_ridge_anchor.append(mean_squared_error(Y_test, Y_pred))
    
    

In [ ]:
df = pd.DataFrame(np.array([score_irm, score_cvp, score_ridge, score, score_ar, score_ridge_anchor, score_pls_ar]).T, columns=['IRM', 'CVP', 'Ridge', 'LR', 'AR', 'A-ridge', 'A-PLS'])
df.boxplot()
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming your score arrays are already defined
df = pd.DataFrame(
    np.array([score_irm, score_cvp, score_ridge, score, score_ar, score_pls_ar]).T, 
    columns=['IRM', 'CVP', 'Ridge', 'LR', 'AR', 'A-PLS']
)

plt.figure(figsize=(8, 6))
sns.violinplot(data=df)
sns.stripplot(data=df, color='black', jitter=True, size=5, alpha=0.7)
plt.ylabel('Mean Squared Error')
plt.xticks(rotation=45)  # Rotate labels for better readability if needed
plt.show()


In [ ]:
# Initialize result dictionary
result = {}

# Reference column for comparison
reference_column = 'LR'

# Determine the algorithm performing best in each row
best_counts = (df.eq(df.min(axis=1), axis=0)).sum()

# Calculate statistics and comparison
for algorithm in df.columns:
    mean_mse = df[algorithm].mean()
    median_mse = df[algorithm].median()
    max_mse = df[algorithm].max()
    min_mse = df[algorithm].min()
    better_than_lr = (df[algorithm] < df[reference_column]).sum()
    best_performance = best_counts[algorithm]

    result[algorithm] = {
        'Mean MSE': mean_mse,
        'Median MSE': median_mse,
        'Max MSE': max_mse,
        'Min MSE': min_mse,
        'Better than LR': better_than_lr    }

# Convert result dictionary to a DataFrame
result_df = pd.DataFrame(result).T

# Convert to markdown
markdown_table = result_df.to_markdown()

print(markdown_table)